In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
import pickle

In [4]:
zomato = pd.read_csv('zomato.csv', encoding='latin-1')

In [6]:
baseReduzida = zomato

In [8]:
baseReduzida.rename(columns={'Average Cost for two':'media_preco',
                          'Price range':'alcance_preco',
                          'Aggregate rating':'classifi_agregada',
                          'Votes':'votos',
                           'Rating text':'taxa_votos',
                          'Cuisines':'cozinha'}, 
                 inplace=True)

In [10]:
baseReduzida = baseReduzida[~baseReduzida["cozinha"].str.contains(" ", na=False)]
baseReduzida.shape

(1679, 21)

In [14]:
baseReduzida.to_csv('basereduzida.csv', index=False)

In [15]:
t = pd.read_csv('basereduzida.csv', encoding='latin-1')

In [19]:
t.rename(columns={'Average Cost for two':'media_preco',
                          'Price range':'alcance_preco',
                          'Aggregate rating':'classifi_agregada',
                          'Votes':'votos',
                           'Rating text':'taxa_votos',
                          'Cuisines':'cozinha'}, 
                 inplace=True)

In [58]:
vai = t.loc[(t['media_preco']) > (230 - 110) & (t['taxa_votos'] == 'Excellent') & (t['cozinha'] == 'Japanese') & (t['votos'] >= (600 - 50)) & (t['alcance_preco'] >= 4)]

In [59]:
vai.shape

(1673, 21)

In [60]:
print(vai.loc[0])

Restaurant ID                                                     6304287
Restaurant Name                                          Izakaya Kikufuji
Country Code                                                          162
City                                                          Makati City
Address                 Little Tokyo, 2277 Chino Roces Avenue, Legaspi...
Locality                       Little Tokyo, Legaspi Village, Makati City
Locality Verbose        Little Tokyo, Legaspi Village, Makati City, Ma...
Longitude                                                         121.014
Latitude                                                          14.5537
cozinha                                                          Japanese
media_preco                                                          1200
Currency                                                 Botswana Pula(P)
Has Table booking                                                     Yes
Has Online delivery                   

In [31]:
i = 0
for i in t.shape[0]:
    print("oi")
    i = i + 1

TypeError: 'int' object is not iterable

In [5]:
df = pd.DataFrame((zomato[['Cuisines', 'Rating text', 'Average Cost for two', 'Price range', 'Aggregate rating', 'Votes']]))

In [172]:
df.head(3)

,Cuisines,Rating text,Average Cost for two,Price range,Aggregate rating,Votes
0,"French, Japanese, Desserts",Excellent,1100,3,4.8,314
1,Japanese,Excellent,1200,3,4.5,591
2,"Seafood, Asian, Filipino, Indian",Very Good,4000,4,4.4,270


In [173]:
df.rename(columns={'Average Cost for two':'media_preco',
                          'Price range':'alcance_preco',
                          'Aggregate rating':'classifi_agregada',
                          'Votes':'votos',
                           'Rating text':'taxa_votos',
                          'Cuisines':'cozinha'}, 
                 inplace=True)

In [174]:
df.head(3)

,cozinha,taxa_votos,media_preco,alcance_preco,classifi_agregada,votos
0,"French, Japanese, Desserts",Excellent,1100,3,4.8,314
1,Japanese,Excellent,1200,3,4.5,591
2,"Seafood, Asian, Filipino, Indian",Very Good,4000,4,4.4,270


In [175]:
dfColunasParaDicionarizar = df[~df["cozinha"].str.contains(" ", na=False)]

In [176]:
ColunaCozinha = dfColunasParaDicionarizar['cozinha']

In [177]:
ColunaCozinha = ColunaCozinha.astype('str')

In [178]:
ColunaTaxaVotos = dfColunasParaDicionarizar['taxa_votos']

In [179]:
ColunaTaxaVotos = ColunaTaxaVotos.astype('str')

In [180]:
X = pd.concat([ColunaCozinha, ColunaTaxaVotos], axis=1)

In [181]:
labelencoder_dict = {}
onehotencoder_dict = {}

X_train = None
for i in range(0, X.shape[1]):
    label_encoder = preprocessing.LabelEncoder()
    labelencoder_dict[i] = label_encoder
    feature = label_encoder.fit_transform(X.iloc[:,i])
    #X_train.iloc[:,:]
    feature = feature.reshape(X.shape[0], 1)
    onehot_encoder = OneHotEncoder(sparse=False)
    feature = onehot_encoder.fit_transform(feature)
    onehotencoder_dict[i] = onehot_encoder
    if X_train is None:
          X_train = feature
    else:
          X_train = np.concatenate((X_train, feature), axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you us

# Salvando dicionário

In [183]:
pickle.dump(labelencoder_dict, open('labelencoder_dict.sav', 'wb'))

In [184]:
pickle.dump(onehotencoder_dict, open('onehotencoder_dict.sav', 'wb'))

# Lendo arquivo de dicionário

In [185]:
labelencoder_dictArquivo = open('labelencoder_dict.sav', 'rb')

In [186]:
labelencoder_dictArquivo = open('onehotencoder_dict.sav', 'rb')

In [114]:
def getEncoded(test_data,labelencoder_dict,onehotencoder_dict):
    test_encoded_x = None
    for i in range(0,test_data.shape[1]):
        label_encoder =  labelencoder_dict[i]
        feature = label_encoder.transform(test_data.iloc[:,i])
        feature = feature.reshape(test_data.shape[0], 1)
        onehot_encoder = onehotencoder_dict[i]
        feature = onehot_encoder.transform(feature)
        if test_encoded_x is None:
              test_encoded_x = feature
        else:
              test_encoded_x = np.concatenate((test_encoded_x, feature), axis=1)
    return test_encoded_x

In [115]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
#X = df['cozinha']

In [116]:
dfProcessado = df[~df["cozinha"].str.contains(" ", na=False)]

In [117]:
dfProcessado.head(2)

,cozinha,taxa_votos,media_preco,alcance_preco,classifi_agregada,votos
1,Japanese,Excellent,1200,3,4.5,591
5,Chinese,Very Good,1000,3,4.4,336


In [118]:
dfProcessado.shape

(1679, 6)

In [119]:
x_treinamentoParte1 = X_train

In [120]:
treinamentoParte2 = pd.DataFrame((dfProcessado[['alcance_preco', 'classifi_agregada', 'votos']]))

In [121]:
treinamentoParte2.head(4)

,alcance_preco,classifi_agregada,votos
1,3,4.5,591
5,3,4.4,336
9,3,4.8,532
13,3,4.2,458


In [122]:
x_treinamentoParte2 = np.array(treinamentoParte2)

In [123]:
X_TRAIN = np.concatenate((x_treinamentoParte1, x_treinamentoParte2), axis=1)

In [124]:
X_TRAIN.shape

(1679, 82)

In [125]:
Y_TRAIN = np.array(dfProcessado.media_preco)

In [126]:
Y_TRAIN.shape

(1679,)

In [127]:
import xgboost as xgb
from sklearn import model_selection

In [128]:
xg_boost = xgb.XGBClassifier()

In [129]:
X_TRAIN.shape

(1679, 82)

In [130]:
Y_TRAIN.shape

(1679,)

In [131]:
xg_boost.fit(X_TRAIN[:100,:], Y_TRAIN[:100])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [132]:
print(xg_boost.score(X_TRAIN, Y_TRAIN))

#scores = model_selection.cross_val_score(xg_boost, X_TRAIN, Y_TRAIN, cv=5, scoring='accuracy')
#print(scores)
#print("Kfold on XGBClassifier: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std()))

0.08695652173913043


In [133]:
# Realizando uma predição

In [134]:
escolha = ['Japanese', 'Excellent', 4, 600, 4]

In [135]:
def escolhaParaVetorPredicao(escolhaLista):
    colunasCategoricasPredicao = [{'cozinha': escolhaLista[0], 'votos': escolhaLista[1]}]
    colunasCategoricasPredicaoDic = pd.DataFrame(colunasCategoricasPredicao)
    return colunasCategoricasPredicaoDic

In [136]:
#colunasCategoricasPredicao = [{'cozinha': escolha[0], 'votos': escolha[1]}]
#colunasCategoricasPredicaoDic = pd.DataFrame(colunasCategoricasPredicao)

In [137]:
dfescolhaParaVetorPredicao = escolhaParaVetorPredicao(escolha)

In [138]:
dicionarioCategoriasPredicaoParte1 = getEncoded(dfescolhaParaVetorPredicao,labelencoder_dict,onehotencoder_dict)

In [139]:
def concatenaParte1e2Dicionario(dicionarioCategoriasPredicaoParte1, escolha):
    PegandoParte2Escolha = np.asarray(escolha[2:])
    Parte2Escolha = PegandoParte2Escolha.reshape(1,3)
    escolha_para_predicao = np.concatenate((dicionarioCategoriasPredicaoParte1, Parte2Escolha), axis=1)
    
    return escolha_para_predicao

In [140]:
dfConcatenadoPraPredicao = concatenaParte1e2Dicionario(dicionarioCategoriasPredicaoParte1, escolha)

In [141]:
print("Média de preço para características buscadas: ", xg_boost.predict(dfConcatenadoPraPredicao))

Média de preço para características buscadas:  [170]


# Salvando o modelo do treinamento

In [158]:
pickle.dump(labelencoder_dict, open('labelencoder_dict.sav', 'wb'))

In [159]:
pickle.dump(onehotencoder_dict, open('onehotencoder_dict.sav', 'wb'))

In [160]:
pickle.dump(xg_boost, open('modeloTreinadoMoveMeTeste.pkl', 'wb'))

# Abrindo o modelo para classificação

In [164]:
labelencoder_dictArquivo = open('labelencoder_dict.sav', 'rb')

In [165]:
onehotencoder_dictArquivo = open('onehotencoder_dict.sav', 'rb')

In [166]:
labelencoder_dict_carregado = pickle.load(labelencoder_dictArquivo)

In [167]:
type(labelencoder_dict_carregado)

dict

In [154]:
arquivo = open('modeloTreinadoMoveMeTeste.pkl', 'rb')

In [155]:
modelo_carregado = pickle.load(arquivo)

In [156]:
numero = modelo_carregado.predict(dfConcatenadoPraPredicao)

# Realizando um predição com o modelo salvo

In [157]:
print("Média de preço predição carregada para características buscadas: ", modelo_carregado.predict(dfConcatenadoPraPredicao))

Média de preço predição carregada para características buscadas:  [170]
